In [5]:
import numpy as np
from scipy.stats import percentileofscore
import scipy.sparse

#users = np.load("./data/netflix_users2_pos=100.npy")
#movies = np.load("./data/netflix_movies2_pos=100.npy")
#vals = np.load("./data/netflix_singular_vals2_pos=100.npy")
#test = scipy.sparse.load_npz("./data/netflix.adjacency.test2.npz")

users = np.load("./data/netflix_users_pmi.npy")
movies = np.load("./data/netflix_movies_pmi.npy")
vals = np.load("./data/netflix_singular_vals_pmi.npy")
train = scipy.sparse.load_npz("./data/netflix.adjacency.train.npz")
test = scipy.sparse.load_npz("./data/netflix.adjacency.test.npz")

# users = np.load("./data/movielens_users_svd.npy")
# movies = np.load("./data/movielens_movies_svd.npy")
# vals = np.load("./data/movielens_singular_vals_svd.npy")

# users = np.load("./data/movielens_users_pos=100.npy")
# movies = np.load("./data/movielens_movies_pos=100.npy")
# vals = np.load("./data/movielens_singular_vals_pos=100.npy")

# train = scipy.sparse.load_npz("./data/movielens.adjacency.train.npz")
# test = scipy.sparse.load_npz("./data/movielens.adjacency.test.npz")

In [28]:
users_path = "./data/netflix.users_w=2.qmf"
items_path = "./data/netflix.items_w=2.qmf"

users_dt = {}

with open(users_path) as f:
    for line in f:
        line = line.strip().split()
        users_dt[int(line[0])] = list(map(float, line[1:]))
        
users = np.zeros([len(users_dt), 100])

for user_id, emb in users_dt.items():
    users[user_id, :] = np.array(emb)

In [29]:
items_dt = {}

with open(items_path) as f:
    for line in f:
        line = line.strip().split()
        items_dt[int(line[0])] = list(map(float, line[1:]))
        
items = np.zeros([max(items_dt.keys()) + 1, 100])

for item_id, emb in items_dt.items():
    items[item_id, :] = np.array(emb)

In [30]:
vals = np.ones(100)
movies = items

In [31]:
#users = users / np.linalg.norm(users, axis=1, keepdims=True)
#movies = movies / np.linalg.norm(movies, axis=1, keepdims=True)

k = 100
user_ids, movie_ids = test.nonzero()
ranks = []
for idx in range(test.shape[0]):
    if idx % 10000 == 0 and idx > 0:
        print(idx, np.mean(ranks))
        break
        
    user_id = user_ids[idx]
    movie_id = movie_ids[idx]
    #sims = movies.dot(users[user_id,:])
    #sims = -np.sum(np.square(movies - users[user_id,:]*vals), axis=1)
    sims = movies[:, -k:].dot(users[user_id,-k:]*vals[-k:])
    rank = percentileofscore(sims, sims[movie_id])
    ranks.append(rank)
    
np.mean(ranks)

10000 93.43588182329769


93.43588182329769

In [32]:
from implicit.evaluation import precision_at_k, mean_average_precision_at_k
import itertools

class SVDModel:
    def __init__(self, users, items, vals):
        self.users = users
        self.item_factors = items
        self.vals = vals
        
    def recommend(self, userid, user_items,
                  N=10, filter_already_liked_items=True, filter_items=None, recalculate_user=False):
        user = self.users[userid] * vals

        # calculate the top N items, removing the users own liked items from the results
        if filter_already_liked_items is True:
            liked = set(user_items[userid].indices)
        else:
            liked = set()
        scores = self.item_factors.dot(user)
        if filter_items:
            liked.update(filter_items)

        count = N + len(liked)
        if count < len(scores):
            ids = np.argpartition(scores, -count)[-count:]
            best = sorted(zip(ids, scores[ids]), key=lambda x: -x[1])
        else:
            best = sorted(enumerate(scores), key=lambda x: -x[1])
        return list(itertools.islice((rec for rec in best if rec[0] not in liked), N))
    
model = SVDModel(users, movies, vals)
pr = precision_at_k(model, train, test, K=20, show_progress=True, num_threads=4)
print(pr)
#mean_pr = mean_average_precision_at_k(model, train, test, K=20, show_progress=True, num_threads=1)
#print(pr, mean_pr)

100%|██████████| 463435/463435 [08:04<00:00, 957.22it/s] 

0.26788654288087865


In [ ]:
### PMI, inner product, neg=1, pos=1
# 10000 88.97819977490151
# 20000 89.64527743387733
# 30000 89.37634758957043
# 40000 88.71577335396735
# 50000 89.0205148002251
# 60000 89.27747008066028
# 70000 89.47819318273173
# 80000 89.78221454698931
# 90000 90.23911392484212
# 100000 90.58637293190772
# 110000 90.84382805545607
# 120000 91.09001716375916
# 130000 91.30192567421325
# 140000 91.50021159257177
# 150000 91.65341410617147
# 160000 91.77862362830615

### PMI, inner product, neg=1, pos=100
# 10000 92.42193528418683
# 20000 92.89404811480021
# 30000 93.04647927218157
# 40000 92.94268036015757
# 50000 93.15109893078221
# 60000 93.36103620333895
# 70000 93.54532293592732
# 80000 93.68624563871693
# 90000 93.85517832801851
# 100000 93.97802476083287
# 110000 94.06193282856704
# 120000 94.147773776027
# 130000 94.2178700922038
# 140000 94.28050852158533
# 150000 94.34195171637592
# 160000 94.37262468345524
# 170000 94.27095094177231
# 180000 94.19613524667041
# 190000 94.14627977371678
# 200000 94.1105785312324
# 210000 94.08751898598491
# 220000 94.09791584386348
# 230000 94.11875207359739
# 240000 94.1738907099981
# 250000 94.19415646595382
# 260000 94.23603292065276
# 270000 94.26528195668936
# 280000 94.26641719591605
# 290000 94.23844705334443
# 300000 94.23204173700992
# 310000 94.2222524552072
# 320000 94.23006037211591
# 330000 94.26428693235107
# 340000 94.25469219437915
# 350000 94.24127701583727
# 360000 94.22227974738946
# 370000 94.21815104412234
# 380000 94.21094077836683
# 390000 94.19272009869701
# 400000 94.18752982554867
# 410000 94.15964744636753
# 420000 94.18837507034326
# 430000 94.20744459567337
# 440000 94.20267831380772
# 450000 94.18832853123239
# 460000 94.17995751266177
# 470000 94.14297981297668

### SVD, inner product
# 10000 92.1537307259426
# 20000 92.64015883511536
# 30000 92.86327565184769
# 40000 92.76784904333145
# 50000 92.97304546989308
# 60000 93.1747843744138
# 70000 93.35709237076937
# 80000 93.50585780106923
# 90000 93.67374748327394
# 100000 93.79049442881261
# 110000 93.87704760321277
# 120000 93.96180662633653
# 130000 94.02646487165057
# 140000 94.08317416994934
# 150000 94.13687936597262
# 160000 94.16676820132245
# 170000 94.04405854215631
# 180000 93.96346728256111

In [ ]:
### Movielens
### PMI, pos=100
# 90.27900618682467

### SVD
# 90.05168813845619